In [3]:
# adesso si pensa all'estrazione delle feature

from tensorflow.keras.applications.inception_v3 import InceptionV3


pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = False

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 34s 0us/step


In [6]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# appiattiamo i layers ad 1 dimensione ok
x = layers.Flatten()(pre_trained_model.output)
# e aggiungiamo un layer fully connected con 1024 hidden units and Relu activation
x = layers.Dense(1024, activation = 'relu')(x)

#dropout rate di 0.2 cosi non perdiamo tempo
x = layers.Dropout(0.2)(x)

x = layers.Dense (1, activation= 'sigmoid') (x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss ='binary_crossentropy',
              metrics = ['acc'])

In [9]:
import os
import random
import shutil

data_path = r"..\data\Flickr8k_Dataset\Images"


train_folder = os.path.join(data_path, 'train')
val_folder = os.path.join(data_path, 'val')
test_folder = os.path.join(data_path, 'test')


image_extensions = ['.jpg']


imgs_list = [filename for filename in os.listdir(data_path) if os.path.splitext(filename)[-1] in image_extensions]

random.seed(42)

random.shuffle(imgs_list)


train_size = int(len(imgs_list) * 0.70)
val_size = int(len(imgs_list) * 0.15)
test_size = int(len(imgs_list) * 0.15)

for folder_path in [train_folder, val_folder, test_folder]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)


for i, f in enumerate(imgs_list):
    if i < train_size:
        dest_folder = train_folder
    elif i < train_size + val_size:
        dest_folder = val_folder
    else:
        dest_folder = test_folder
    shutil.copy(os.path.join(data_path, f), os.path.join(dest_folder, f))

In [12]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np
import os
import glob
import tqdm
import pickle

# Modello CNN pre-addestrato (InceptionV3)
base_model = InceptionV3(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

# Funzione per estrarre le feature da una singola immagine
def extract_feature(img_path, model):
    img = image.load_img(img_path, target_size=(299, 299))  # Inception richiede questa dimensione
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x, verbose=0)
    return feature.reshape(2048,)

# Percorso delle immagini
images_path = "../data/Flickr8k_Dataset/Images"
image_files = glob.glob(os.path.join(images_path, "*.jpg"))

# Crea cartella per salvare le feature
os.makedirs("features", exist_ok=True)

# Estrazione delle feature per tutte le immagini
features = {}
for img_path in tqdm.tqdm(image_files):
    image_id = os.path.basename(img_path).split('.')[0]
    features[image_id] = extract_feature(img_path, model)

# Salvataggio su file
with open("features/image_features.pkl", "wb") as f:
    pickle.dump(features, f)

print("✔️ Estrazione delle feature completata.")


96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 31s 0us/step


100%|██████████| 8091/8091 [27:58<00:00,  4.82it/s]  


✔️ Estrazione delle feature completata.
